In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
import cv2

from openslide import OpenSlide
from pathlib import Path
from scipy.ndimage import binary_erosion, binary_dilation, gaussian_filter
from PIL import Image
from tqdm.notebook import tqdm

r = Path('/home/rens/hpc/PREMIUM histopathology/data/mst/metastasis')
annotation_path = Path('/home/rens/hpc/rens/annotations/mst/metastasis')
dest_folder = Path('/home/rens/hpc/rens/output/patches/mst_metastasis_level4')

for file in tqdm(sorted(list(r.iterdir()))):
    roi_file = annotation_path / (file.stem + '.xml')
    try:
        with open(roi_file, 'r') as f:
                xml_file = ET.parse(f)
    except:
        # print('no annotation')
        continue

    slide = OpenSlide(str(file))
    # img = np.array(slide.read_region((0,0), 4, slide.level_dimensions[4]).convert('RGB'))

    # plt.figure(figsize=(12,12))
    # plt.imshow(img)
    # plt.show()

    mag_factor = int(slide.level_dimensions[0][0] / slide.level_dimensions[4][0])

    # extract all rois
    rois = []
    for annotation in xml_file.getroot()[0]:
        roi = []
        for coordinate in annotation[0]:
            roi.append([coordinate.attrib['X'], coordinate.attrib['Y']])
        rois.append((np.array(roi).astype(float) / mag_factor).astype(int))

    mask = np.zeros(np.flip(slide.level_dimensions[4]))
    cv2.fillPoly(mask, rois, 1)

    # mask = img.sum(axis=-1) < 650
    # eroded = binary_erosion(mask, iterations=25)
    # dilated = binary_dilation(eroded, iterations=20)

    # plt.figure(figsize=(12,12))
    # plt.imshow(mask)
    # plt.show()

    patch_size = 256

    xx = np.arange(0, mask.shape[0] - mask.shape[0] % patch_size, patch_size)
    yy = np.arange(0, mask.shape[1] - mask.shape[1] % patch_size, patch_size)
    # xx = np.arange(0, mask.shape[0] - patch_size, patch_size - 96)
    # yy = np.arange(0, mask.shape[0] - patch_size, patch_size - 96)

    origins = []
    for x in xx:
        for y in yy:
            if mask[x:x+patch_size,y:y+patch_size].sum() > 0:
                origins.append((x,y))

    # plt.figure(figsize=(12,12))
    # plt.imshow(img)
    # for x,y in origins:
    #     plt.plot(
    #         [y,y+patch_size,y+patch_size,y,y],
    #         [x,x,x+patch_size, x+patch_size,x],
    #         c='b'
    #     )
    # plt.show()
    for x,y in origins:
        # Image.fromarray(img[x:x+patch_size, y:y+patch_size]).save(dest_folder / (file.stem + f"_x{x}_y{y}.jpeg"))
        Image.fromarray(
            # img[x:x+patch_size, y:y+patch_size]
            np.array(slide.read_region(
                (y * mag_factor,x * mag_factor), 4, 2*[patch_size]
            ).convert('RGB'))
        ).save(dest_folder / (file.stem + f"_x{x}_y{y}.jpeg"))

  0%|          | 0/57 [00:00<?, ?it/s]